In [293]:
import numpy as np
import torch
import torch.nn as nn
import requests

In [383]:
def readQAP_from_URL(url):
    """
    Lee un archivo DAT desde una URL y lo transforma en un diccionario que contiene el número de nodos (n), matriz de flujo (f) 
    y matriz de distancia (d)

    Args:
        url (str): URL del archivo .dat a leer

    Returns:
        instance_dict (dict): Diccionario que contiene el número de nodos (n), matriz de flujo (f) y matriz de distancia (d)
    """
    response = requests.get(url)
    content = response.text.split('\n')
    
    n_nodos = int(content[0])
    flow_matrix = np.array([list(map(int, line.split())) for line in content[1:n_nodos+1]])
    distance_matrix = np.array([list(map(int, line.split())) for line in content[n_nodos+2:2*n_nodos+2]])
    
    instance_dict = {'n': n_nodos, 'f': flow_matrix, 'd': distance_matrix}
    
    return instance_dict


def evaluateQAP(sol, f, d):
    """
    Evaluar QAP con una solución dada

    Args:
        sol (numpy array): vector con valores de posición para evaluar la función
        f (numpy array): matriz de flujo entre nodos
        d (numpy array): matriz de distancia entre nodos

    Returns:
        Valor de fitness del QAP en base a la solución entregada
    """
    
    fitness = np.sum(f * d[sol[:, None], sol])
    
    return fitness


def generate_population(size, n):
    return [np.random.permutation(n) for _ in range(size)]

def fitness(solution, instance):
    return evaluateQAP(solution, instance['f'], instance['d'])


def crossover(parent1, parent2):
    # Perform ordered crossover
    size = len(parent1)
    idx1, idx2 = sorted(np.random.choice(range(size), 2, replace=False))
    child = -np.ones(size, dtype=int)
    child[idx1:idx2] = parent1[idx1:idx2]
    mask = np.isin(parent2, child)
    child[child==-1] = parent2[~mask]
    return child

def selection(population, scores, k=3):
    # Select k random individuals from the population and return the best one
    selection_idx = np.random.choice(len(population), k, replace=False)
    return population[min(selection_idx, key=lambda idx: scores[idx])]

def mutation(solution):
    # Perform swap mutation
    idx1, idx2 = np.random.choice(range(len(solution)), 2, replace=False)
    solution[idx1], solution[idx2] = solution[idx2], solution[idx1]
    return solution

def replace_with_new_population(population, new_population, scores):
    # Replace the old population with the new population
    population_size = len(population)
    new_scores = [fitness(solution, instance) for solution in new_population]
    population = population + new_population
    scores = scores + new_scores
    idx = np.argsort(scores)[:population_size]
    return [population[i] for i in idx], [scores[i] for i in idx]


def population_diversity(population):
    # A simple measure of population diversity is the standard deviation of the solutions
    return np.std([np.sum(sol) for sol in population])


def genetic_algorithm(instance, n_population, n_iterations, mutation_rate):
    population = generate_population(n_population, instance['n'])
    scores = [fitness(solution, instance) for solution in population]

    for _ in range(n_iterations):
        
        print("Iteración ", _)
        
        new_population = []
        for _ in range(n_population):
            parent1 = selection(population, scores)
            parent2 = selection(population, scores)
            child = crossover(parent1, parent2)
            if np.random.rand() < mutation_rate:
                child = mutation(child)
            new_population.append(child)
        population, scores = replace_with_new_population(population, new_population, scores)
        
        print("Menor Costo:", min(scores))
        
    
    best_solution = population[np.argmin(scores)]
    best_score = min(scores)
    
    
    
    return best_solution, best_score


class Actor(nn.Module):
    def __init__(self):
        super(Actor, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(3, 64),  # Cambia la dimensión del estado a 3
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()  # Sigmoid para que la tasa de mutación esté entre 0 y 1
        )

    def forward(self, state):
        return self.network(state)



class Critic(nn.Module):
    def __init__(self):
        super(Critic, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(3, 64),  # Cambia la dimensión del estado a 3
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, state):
        return self.network(state)

    
def genetic_algorithm_ac(instance, n_population, n_iterations, actor, critic, actor_optimizer, critic_optimizer):
    population = generate_population(n_population, instance['n'])
    scores = [fitness(solution, instance) for solution in population]

    # Define some parameters for noise decay
    initial_noise_scale = 0.5
    noise_decay = 0.995
    noise_scale = initial_noise_scale

    for i in range(n_iterations):
        
        print("Iteración ", i)
        
        actor_optimizer.zero_grad()
        critic_optimizer.zero_grad()
        
        new_population = []
        new_scores = []  # Track scores of the new population
        rewards = []  # Track rewards for each child
        
        # Decide mutation rate with some noise added
        state = torch.tensor([np.mean(scores), population_diversity(population), i]).float()
        mutation_rate = actor(state)
        noise = torch.randn_like(mutation_rate) * noise_scale
        mutation_rate = torch.clamp(mutation_rate + noise, 0, 1)  # Make sure mutation_rate is between 0 and 1
        
        print("Ratio de mutación: ", mutation_rate.item())
        
        last_score = min(scores)  # Update last_score at the beginning of the generation
        
        for _ in range(n_population):
            parent1 = selection(population, scores)
            parent2 = selection(population, scores)
            child = crossover(parent1, parent2)

            if np.random.rand() < mutation_rate:
                child = mutation(child)

            new_population.append(child)
            child_score = fitness(child, instance)
            new_scores.append(child_score)  # Compute the score of the new child
            
            # Compute individual reward
            rewards.append(last_score - child_score)
            
            print("Recompensa Pasada: ", last_score, "Recompensa Actual: ", child_score, "Recompensa:", last_score - child_score)

        # Compute average reward
        reward = np.max(rewards)
        print("Recompensa: ", reward)

        # Compute the new state after the entire new population has been generated
        state = torch.tensor([np.mean(new_scores), population_diversity(new_population), i]).float()

        # Update Critic
        target_value = reward + critic(state)  # Target value is based on reward and current value estimate
        critic_loss = (target_value - critic(state))**2
        critic_loss.backward()
        critic_optimizer.step()
        critic_optimizer.zero_grad()  # Reset gradients

        # Update Actor
        actor_loss = -critic(state)  # We want to maximize the value estimate        
        actor_loss.backward()
        actor_optimizer.step()
        actor_optimizer.zero_grad()  # Reset gradients for the next iteration

        # Replace the old population with the new one
        population, scores = new_population, new_scores

        # Decrease noise scale
        noise_scale *= noise_decay

    best_solution = population[np.argmin(scores)]
    best_score = min(scores)
    
    return best_solution, best_score


# Entradas

In [336]:
instance = readQAP_from_URL("https://www.opt.math.tugraz.at/qaplib/data.d/tho40.dat")

# Parámetros

In [347]:
n_population = 20
n_iterations = 1500
mutation_rate= 0.9

# Genetic Algorithm

In [371]:
results_ga = genetic_algorithm(instance, n_population, n_iterations, mutation_rate)

Iteración  0
Menor Costo: 320726
Iteración  1
Menor Costo: 318938
Iteración  2
Menor Costo: 307470
Iteración  3
Menor Costo: 307470
Iteración  4
Menor Costo: 304086
Iteración  5
Menor Costo: 304086
Iteración  6
Menor Costo: 304074
Iteración  7
Menor Costo: 302884
Iteración  8
Menor Costo: 302884
Iteración  9
Menor Costo: 294670
Iteración  10
Menor Costo: 294670
Iteración  11
Menor Costo: 294670
Iteración  12
Menor Costo: 294670
Iteración  13
Menor Costo: 294670
Iteración  14
Menor Costo: 294670
Iteración  15
Menor Costo: 294670
Iteración  16
Menor Costo: 294356
Iteración  17
Menor Costo: 293088
Iteración  18
Menor Costo: 293088
Iteración  19
Menor Costo: 291334
Iteración  20
Menor Costo: 290884
Iteración  21
Menor Costo: 290250
Iteración  22
Menor Costo: 289762
Iteración  23
Menor Costo: 287988
Iteración  24
Menor Costo: 287988
Iteración  25
Menor Costo: 287464
Iteración  26
Menor Costo: 287464
Iteración  27
Menor Costo: 287464
Iteración  28
Menor Costo: 287464
Iteración  29
Menor Cost

Menor Costo: 247414
Iteración  305
Menor Costo: 247414
Iteración  306
Menor Costo: 247414
Iteración  307
Menor Costo: 247414
Iteración  308
Menor Costo: 247414
Iteración  309
Menor Costo: 247414
Iteración  310
Menor Costo: 247414
Iteración  311
Menor Costo: 247414
Iteración  312
Menor Costo: 247414
Iteración  313
Menor Costo: 247414
Iteración  314
Menor Costo: 246998
Iteración  315
Menor Costo: 246998
Iteración  316
Menor Costo: 246642
Iteración  317
Menor Costo: 246642
Iteración  318
Menor Costo: 246642
Iteración  319
Menor Costo: 246642
Iteración  320
Menor Costo: 246642
Iteración  321
Menor Costo: 246258
Iteración  322
Menor Costo: 246258
Iteración  323
Menor Costo: 246258
Iteración  324
Menor Costo: 246258
Iteración  325
Menor Costo: 246258
Iteración  326
Menor Costo: 246258
Iteración  327
Menor Costo: 246258
Iteración  328
Menor Costo: 246258
Iteración  329
Menor Costo: 246258
Iteración  330
Menor Costo: 246258
Iteración  331
Menor Costo: 246258
Iteración  332
Menor Costo: 246258


Menor Costo: 244666
Iteración  612
Menor Costo: 244666
Iteración  613
Menor Costo: 244666
Iteración  614
Menor Costo: 244666
Iteración  615
Menor Costo: 244666
Iteración  616
Menor Costo: 244666
Iteración  617
Menor Costo: 244666
Iteración  618
Menor Costo: 244666
Iteración  619
Menor Costo: 244666
Iteración  620
Menor Costo: 244666
Iteración  621
Menor Costo: 244666
Iteración  622
Menor Costo: 244666
Iteración  623
Menor Costo: 244666
Iteración  624
Menor Costo: 244666
Iteración  625
Menor Costo: 244666
Iteración  626
Menor Costo: 244666
Iteración  627
Menor Costo: 244666
Iteración  628
Menor Costo: 244666
Iteración  629
Menor Costo: 244666
Iteración  630
Menor Costo: 244666
Iteración  631
Menor Costo: 244666
Iteración  632
Menor Costo: 244666
Iteración  633
Menor Costo: 244666
Iteración  634
Menor Costo: 244666
Iteración  635
Menor Costo: 244666
Iteración  636
Menor Costo: 244666
Iteración  637
Menor Costo: 244666
Iteración  638
Menor Costo: 244666
Iteración  639
Menor Costo: 244666


Menor Costo: 244666
Iteración  917
Menor Costo: 244666
Iteración  918
Menor Costo: 244666
Iteración  919
Menor Costo: 244666
Iteración  920
Menor Costo: 244666
Iteración  921
Menor Costo: 244666
Iteración  922
Menor Costo: 244666
Iteración  923
Menor Costo: 244666
Iteración  924
Menor Costo: 244666
Iteración  925
Menor Costo: 244666
Iteración  926
Menor Costo: 244666
Iteración  927
Menor Costo: 244666
Iteración  928
Menor Costo: 244666
Iteración  929
Menor Costo: 244666
Iteración  930
Menor Costo: 244666
Iteración  931
Menor Costo: 244666
Iteración  932
Menor Costo: 244666
Iteración  933
Menor Costo: 244666
Iteración  934
Menor Costo: 244666
Iteración  935
Menor Costo: 244666
Iteración  936
Menor Costo: 244666
Iteración  937
Menor Costo: 244666
Iteración  938
Menor Costo: 244666
Iteración  939
Menor Costo: 244666
Iteración  940
Menor Costo: 244666
Iteración  941
Menor Costo: 244666
Iteración  942
Menor Costo: 244666
Iteración  943
Menor Costo: 244666
Iteración  944
Menor Costo: 244666


Menor Costo: 244666
Iteración  1148
Menor Costo: 244666
Iteración  1149
Menor Costo: 244666
Iteración  1150
Menor Costo: 244666
Iteración  1151
Menor Costo: 244666
Iteración  1152
Menor Costo: 244666
Iteración  1153
Menor Costo: 244666
Iteración  1154
Menor Costo: 244666
Iteración  1155
Menor Costo: 244666
Iteración  1156
Menor Costo: 244666
Iteración  1157
Menor Costo: 244666
Iteración  1158
Menor Costo: 244666
Iteración  1159
Menor Costo: 244666
Iteración  1160
Menor Costo: 244666
Iteración  1161
Menor Costo: 244666
Iteración  1162
Menor Costo: 244666
Iteración  1163
Menor Costo: 244666
Iteración  1164
Menor Costo: 244666
Iteración  1165
Menor Costo: 244666
Iteración  1166
Menor Costo: 244666
Iteración  1167
Menor Costo: 244666
Iteración  1168
Menor Costo: 244666
Iteración  1169
Menor Costo: 244666
Iteración  1170
Menor Costo: 244666
Iteración  1171
Menor Costo: 244666
Iteración  1172
Menor Costo: 244666
Iteración  1173
Menor Costo: 244666
Iteración  1174
Menor Costo: 244666
Iteració

Menor Costo: 244666
Iteración  1376
Menor Costo: 244666
Iteración  1377
Menor Costo: 244666
Iteración  1378
Menor Costo: 244666
Iteración  1379
Menor Costo: 244666
Iteración  1380
Menor Costo: 244666
Iteración  1381
Menor Costo: 244666
Iteración  1382
Menor Costo: 244666
Iteración  1383
Menor Costo: 244666
Iteración  1384
Menor Costo: 244666
Iteración  1385
Menor Costo: 244666
Iteración  1386
Menor Costo: 244666
Iteración  1387
Menor Costo: 244666
Iteración  1388
Menor Costo: 244666
Iteración  1389
Menor Costo: 244666
Iteración  1390
Menor Costo: 244666
Iteración  1391
Menor Costo: 244666
Iteración  1392
Menor Costo: 244666
Iteración  1393
Menor Costo: 244666
Iteración  1394
Menor Costo: 244666
Iteración  1395
Menor Costo: 244666
Iteración  1396
Menor Costo: 244666
Iteración  1397
Menor Costo: 244666
Iteración  1398
Menor Costo: 244666
Iteración  1399
Menor Costo: 244666
Iteración  1400
Menor Costo: 244666
Iteración  1401
Menor Costo: 244666
Iteración  1402
Menor Costo: 244666
Iteració

In [349]:
results_ga

(array([ 6, 18, 39, 13,  1, 23, 20, 22, 35, 11,  8,  0, 29, 27, 12, 36,  2,
        24,  7,  5, 31, 32, 26,  3, 33, 28, 10, 14, 21, 15, 34,  4, 16, 30,
         9, 19, 38, 17, 25, 37]),
 249488)

# G.A con Actor-Critic

In [384]:
actor = Actor()
critic = Critic()

actor_optimizer = torch.optim.Adam(actor.parameters())
critic_optimizer = torch.optim.Adam(critic.parameters())

In [385]:
best_solution, best_score = genetic_algorithm_ac(instance, n_population, n_iterations, actor, critic, actor_optimizer, critic_optimizer)

Iteración  0
Ratio de mutación:  0.6382222175598145
Recompensa Pasada:  331228 Recompensa Actual:  338822 Recompensa: -7594
Recompensa Pasada:  331228 Recompensa Actual:  326626 Recompensa: 4602
Recompensa Pasada:  331228 Recompensa Actual:  341958 Recompensa: -10730
Recompensa Pasada:  331228 Recompensa Actual:  337830 Recompensa: -6602
Recompensa Pasada:  331228 Recompensa Actual:  339832 Recompensa: -8604
Recompensa Pasada:  331228 Recompensa Actual:  338182 Recompensa: -6954
Recompensa Pasada:  331228 Recompensa Actual:  345814 Recompensa: -14586
Recompensa Pasada:  331228 Recompensa Actual:  343528 Recompensa: -12300
Recompensa Pasada:  331228 Recompensa Actual:  340420 Recompensa: -9192
Recompensa Pasada:  331228 Recompensa Actual:  342476 Recompensa: -11248
Recompensa Pasada:  331228 Recompensa Actual:  332418 Recompensa: -1190
Recompensa Pasada:  331228 Recompensa Actual:  341692 Recompensa: -10464
Recompensa Pasada:  331228 Recompensa Actual:  357226 Recompensa: -25998
Recompe

Iteración  33
Ratio de mutación:  1.0
Recompensa Pasada:  294838 Recompensa Actual:  295742 Recompensa: -904
Recompensa Pasada:  294838 Recompensa Actual:  307378 Recompensa: -12540
Recompensa Pasada:  294838 Recompensa Actual:  311104 Recompensa: -16266
Recompensa Pasada:  294838 Recompensa Actual:  305520 Recompensa: -10682
Recompensa Pasada:  294838 Recompensa Actual:  308232 Recompensa: -13394
Recompensa Pasada:  294838 Recompensa Actual:  326228 Recompensa: -31390
Recompensa Pasada:  294838 Recompensa Actual:  315076 Recompensa: -20238
Recompensa Pasada:  294838 Recompensa Actual:  307074 Recompensa: -12236
Recompensa Pasada:  294838 Recompensa Actual:  304002 Recompensa: -9164
Recompensa Pasada:  294838 Recompensa Actual:  335804 Recompensa: -40966
Recompensa Pasada:  294838 Recompensa Actual:  324122 Recompensa: -29284
Recompensa Pasada:  294838 Recompensa Actual:  300864 Recompensa: -6026
Recompensa Pasada:  294838 Recompensa Actual:  326200 Recompensa: -31362
Recompensa Pasada

Recompensa Pasada:  303024 Recompensa Actual:  318384 Recompensa: -15360
Recompensa Pasada:  303024 Recompensa Actual:  317842 Recompensa: -14818
Recompensa Pasada:  303024 Recompensa Actual:  314518 Recompensa: -11494
Recompensa Pasada:  303024 Recompensa Actual:  313732 Recompensa: -10708
Recompensa:  1406
Iteración  70
Ratio de mutación:  0.7327409982681274
Recompensa Pasada:  301618 Recompensa Actual:  312940 Recompensa: -11322
Recompensa Pasada:  301618 Recompensa Actual:  320460 Recompensa: -18842
Recompensa Pasada:  301618 Recompensa Actual:  321020 Recompensa: -19402
Recompensa Pasada:  301618 Recompensa Actual:  317720 Recompensa: -16102
Recompensa Pasada:  301618 Recompensa Actual:  320202 Recompensa: -18584
Recompensa Pasada:  301618 Recompensa Actual:  318218 Recompensa: -16600
Recompensa Pasada:  301618 Recompensa Actual:  329608 Recompensa: -27990
Recompensa Pasada:  301618 Recompensa Actual:  334190 Recompensa: -32572
Recompensa Pasada:  301618 Recompensa Actual:  323594

Ratio de mutación:  1.0
Recompensa Pasada:  295428 Recompensa Actual:  303236 Recompensa: -7808
Recompensa Pasada:  295428 Recompensa Actual:  324034 Recompensa: -28606
Recompensa Pasada:  295428 Recompensa Actual:  313518 Recompensa: -18090
Recompensa Pasada:  295428 Recompensa Actual:  311692 Recompensa: -16264
Recompensa Pasada:  295428 Recompensa Actual:  301134 Recompensa: -5706
Recompensa Pasada:  295428 Recompensa Actual:  312698 Recompensa: -17270
Recompensa Pasada:  295428 Recompensa Actual:  338016 Recompensa: -42588
Recompensa Pasada:  295428 Recompensa Actual:  317750 Recompensa: -22322
Recompensa Pasada:  295428 Recompensa Actual:  294882 Recompensa: 546
Recompensa Pasada:  295428 Recompensa Actual:  325242 Recompensa: -29814
Recompensa Pasada:  295428 Recompensa Actual:  323214 Recompensa: -27786
Recompensa Pasada:  295428 Recompensa Actual:  314190 Recompensa: -18762
Recompensa Pasada:  295428 Recompensa Actual:  300054 Recompensa: -4626
Recompensa Pasada:  295428 Recomp

Recompensa Pasada:  307346 Recompensa Actual:  322270 Recompensa: -14924
Recompensa Pasada:  307346 Recompensa Actual:  320708 Recompensa: -13362
Recompensa Pasada:  307346 Recompensa Actual:  324440 Recompensa: -17094
Recompensa Pasada:  307346 Recompensa Actual:  312528 Recompensa: -5182
Recompensa Pasada:  307346 Recompensa Actual:  314694 Recompensa: -7348
Recompensa Pasada:  307346 Recompensa Actual:  327630 Recompensa: -20284
Recompensa Pasada:  307346 Recompensa Actual:  339914 Recompensa: -32568
Recompensa Pasada:  307346 Recompensa Actual:  307346 Recompensa: 0
Recompensa Pasada:  307346 Recompensa Actual:  317580 Recompensa: -10234
Recompensa Pasada:  307346 Recompensa Actual:  315904 Recompensa: -8558
Recompensa Pasada:  307346 Recompensa Actual:  315890 Recompensa: -8544
Recompensa Pasada:  307346 Recompensa Actual:  323942 Recompensa: -16596
Recompensa Pasada:  307346 Recompensa Actual:  306594 Recompensa: 752
Recompensa Pasada:  307346 Recompensa Actual:  310982 Recompens

...

Recompensa Pasada:  314132 Recompensa Actual:  315088 Recompensa: -956
Recompensa Pasada:  314132 Recompensa Actual:  323538 Recompensa: -9406
Recompensa Pasada:  314132 Recompensa Actual:  328730 Recompensa: -14598
Recompensa Pasada:  314132 Recompensa Actual:  326380 Recompensa: -12248
Recompensa Pasada:  314132 Recompensa Actual:  334002 Recompensa: -19870
Recompensa Pasada:  314132 Recompensa Actual:  322690 Recompensa: -8558
Recompensa Pasada:  314132 Recompensa Actual:  350460 Recompensa: -36328
Recompensa Pasada:  314132 Recompensa Actual:  330516 Recompensa: -16384
Recompensa Pasada:  314132 Recompensa Actual:  313188 Recompensa: 944
Recompensa Pasada:  314132 Recompensa Actual:  325860 Recompensa: -11728
Recompensa Pasada:  314132 Recompensa Actual:  323258 Recompensa: -9126
Recompensa Pasada:  314132 Recompensa Actual:  309870 Recompensa: 4262
Recompensa:  4262
Iteración  1377
Ratio de mutación:  0.999907910823822
Recompensa Pasada:  309870 Recompensa Actual:  315934 Recompen

Recompensa Pasada:  303070 Recompensa Actual:  322686 Recompensa: -19616
Recompensa Pasada:  303070 Recompensa Actual:  330528 Recompensa: -27458
Recompensa Pasada:  303070 Recompensa Actual:  322742 Recompensa: -19672
Recompensa Pasada:  303070 Recompensa Actual:  321194 Recompensa: -18124
Recompensa Pasada:  303070 Recompensa Actual:  299094 Recompensa: 3976
Recompensa Pasada:  303070 Recompensa Actual:  320108 Recompensa: -17038
Recompensa Pasada:  303070 Recompensa Actual:  316892 Recompensa: -13822
Recompensa:  3976
Iteración  1404
Ratio de mutación:  0.9998151659965515
Recompensa Pasada:  299094 Recompensa Actual:  324578 Recompensa: -25484
Recompensa Pasada:  299094 Recompensa Actual:  306998 Recompensa: -7904
Recompensa Pasada:  299094 Recompensa Actual:  314388 Recompensa: -15294
Recompensa Pasada:  299094 Recompensa Actual:  303794 Recompensa: -4700
Recompensa Pasada:  299094 Recompensa Actual:  331624 Recompensa: -32530
Recompensa Pasada:  299094 Recompensa Actual:  324082 R

Recompensa Pasada:  311800 Recompensa Actual:  319004 Recompensa: -7204
Recompensa Pasada:  311800 Recompensa Actual:  327472 Recompensa: -15672
Recompensa Pasada:  311800 Recompensa Actual:  312922 Recompensa: -1122
Recompensa Pasada:  311800 Recompensa Actual:  311198 Recompensa: 602
Recompensa Pasada:  311800 Recompensa Actual:  338804 Recompensa: -27004
Recompensa Pasada:  311800 Recompensa Actual:  319072 Recompensa: -7272
Recompensa Pasada:  311800 Recompensa Actual:  317654 Recompensa: -5854
Recompensa Pasada:  311800 Recompensa Actual:  314826 Recompensa: -3026
Recompensa Pasada:  311800 Recompensa Actual:  321680 Recompensa: -9880
Recompensa Pasada:  311800 Recompensa Actual:  328144 Recompensa: -16344
Recompensa Pasada:  311800 Recompensa Actual:  321400 Recompensa: -9600
Recompensa Pasada:  311800 Recompensa Actual:  323836 Recompensa: -12036
Recompensa Pasada:  311800 Recompensa Actual:  318172 Recompensa: -6372
Recompensa Pasada:  311800 Recompensa Actual:  331410 Recompen

Recompensa Pasada:  307502 Recompensa Actual:  342768 Recompensa: -35266
Recompensa Pasada:  307502 Recompensa Actual:  323850 Recompensa: -16348
Recompensa Pasada:  307502 Recompensa Actual:  332464 Recompensa: -24962
Recompensa Pasada:  307502 Recompensa Actual:  325776 Recompensa: -18274
Recompensa Pasada:  307502 Recompensa Actual:  308730 Recompensa: -1228
Recompensa Pasada:  307502 Recompensa Actual:  321078 Recompensa: -13576
Recompensa Pasada:  307502 Recompensa Actual:  335590 Recompensa: -28088
Recompensa Pasada:  307502 Recompensa Actual:  320900 Recompensa: -13398
Recompensa Pasada:  307502 Recompensa Actual:  314342 Recompensa: -6840
Recompensa Pasada:  307502 Recompensa Actual:  323978 Recompensa: -16476
Recompensa Pasada:  307502 Recompensa Actual:  328376 Recompensa: -20874
Recompensa Pasada:  307502 Recompensa Actual:  315532 Recompensa: -8030
Recompensa Pasada:  307502 Recompensa Actual:  304440 Recompensa: 3062
Recompensa:  3062
Iteración  1459
Ratio de mutación:  0.

Iteración  1484
Ratio de mutación:  1.0
Recompensa Pasada:  314500 Recompensa Actual:  316696 Recompensa: -2196
Recompensa Pasada:  314500 Recompensa Actual:  321790 Recompensa: -7290
Recompensa Pasada:  314500 Recompensa Actual:  328814 Recompensa: -14314
Recompensa Pasada:  314500 Recompensa Actual:  324862 Recompensa: -10362
Recompensa Pasada:  314500 Recompensa Actual:  323536 Recompensa: -9036
Recompensa Pasada:  314500 Recompensa Actual:  320800 Recompensa: -6300
Recompensa Pasada:  314500 Recompensa Actual:  329890 Recompensa: -15390
Recompensa Pasada:  314500 Recompensa Actual:  340890 Recompensa: -26390
Recompensa Pasada:  314500 Recompensa Actual:  331090 Recompensa: -16590
Recompensa Pasada:  314500 Recompensa Actual:  322602 Recompensa: -8102
Recompensa Pasada:  314500 Recompensa Actual:  336712 Recompensa: -22212
Recompensa Pasada:  314500 Recompensa Actual:  342452 Recompensa: -27952
Recompensa Pasada:  314500 Recompensa Actual:  324616 Recompensa: -10116
Recompensa Pasad

In [386]:
best_solution

array([17,  6,  3, 12, 24, 34,  4, 28, 29, 26, 15,  5, 31, 23, 35, 14, 32,
       37,  7, 13, 21,  1, 39, 30, 33, 38, 27, 16, 11, 10,  2,  0,  8, 36,
       22, 20, 25,  9, 19, 18])

In [387]:
best_score

313598